In [ ]:
import pandas as pd
from src.constants import (
    PREPROCESSING_PATH,
    TARGET
)

import numpy as np
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import power_transform
from utils.classes.extractor import FunctionalGroupToGramExtractor

from typing import (
    Optional,
    List,
    Any
)
from sklearn.model_selection import KFold, RepeatedKFold, RepeatedStratifiedKFold

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score




Train = pd.read_csv(f"{PREPROCESSING_PATH}/train_merged_CIF/Train.csv")
Train = Train.set_index(Train.columns[0])
Train = Train.replace([np.inf], 99999999)
Train = Train.replace([-np.inf], -99999999)
Train = Train.dropna()


Train = Train.drop(Train[
    (Train["surface_area [m^2/g]"]<=0) | (Train["void_fraction"] == 0)
].index)
Train['density'] = (Train["weight [u]"] / Train["volume [A^3]"]) * 1.66054
Train.sample(frac=1, random_state=1234)
ratio = 0.7

train = Train.iloc[: int(ratio * Train.shape[0])]

test = Train.iloc[int(ratio * Train.shape[0]):]



X = train[
    [
        "void_fraction",
        'void_volume [cm^3/g]',
        "heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]",
        "CO2/N2_selectivity",
        "_cell_volume",
        "_cell_length_a",
        "_cell_length_b",
        "_cell_length_c",
        "density"
    ]
]

y = train[['surface_area [m^2/g]']]

X_test = test[
    [
        "void_fraction",
        "void_volume [cm^3/g]",
        "heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]",
        "CO2/N2_selectivity",
        "_cell_volume",
        "_cell_length_a",
        "_cell_length_b",
        "_cell_length_c",
        "density"
    ]
]

y_test = test[['surface_area [m^2/g]']]
# y_test = test["surface_area [m^2/g]"]  # **0.5]

In [ ]:
from xgboost import XGBRegressor

reg = XGBRegressor(grow_policy="lossguide", num_parallel_tree=1)
reg.fit(X.values, y.values)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             grow_policy='lossguide', importance_type='gain',
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [ ]:
mean_absolute_error(reg.predict(X_test), y_test)

230.01658035643382

In [ ]:
pd.DataFrame(reg.predict(X_test))

,0
0,2814.431396
1,1716.003662
2,2338.002930
3,782.264954
4,3158.952148
...,...
16129,1154.280518
16130,4670.764160
16131,3823.479492
16132,4814.241211


In [ ]:
pd.DataFrame(y_test)

,surface_area [m^2/g]
Unnamed: 0,
47630,2735.72
47631,1647.50
47632,2478.84
47633,389.34
47635,2762.68
...,...
66520,1343.62
66521,4182.24
66522,4149.64


In [ ]:
X_final = Train[
    [
        "void_fraction",
        'void_volume [cm^3/g]',
        "heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]",
        "CO2/N2_selectivity",
        "_cell_volume",
        "_cell_length_a",
        "_cell_length_b",
        "_cell_length_c",
        "density"
    ]
]

y_final = Train[['surface_area [m^2/g]']]


reg = XGBRegressor(max_depth=6, grow_policy="lossguide", num_parallel_tree=1)
reg.fit(X_final.values, y_final.values)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             grow_policy='lossguide', importance_type='gain',
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [ ]:
mean_absolute_error(reg.predict(X_final), y_final)

97.37195821579785

In [ ]:
reg.save_model("imputer/xgboost/surface_area.json")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cf8541de-dbc3-45f6-bc1e-4fa446cacbcd' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>